In [3]:
from sys import path
import pandas as pd

path.append(r'C:\Users\Izogie\Desktop\Folders\Projects\Python\KB Chat\src')
from modules.SourceManager import SourceManager

In [4]:
manager = SourceManager()
data = manager.load_json("./articles.jsonl")

In [41]:
df = pd.DataFrame.from_dict(data)
df

,title,links,sections
0,Kaladin,"[Assassin in White, Katarotam, Hobber, Fen Rna...","[{'title': 'Kaladin', 'content': 'Kaladin, nic..."
1,Pits of Hathsin,"[Final Ascension, ref-wob-5525, Ruin's Perpend...","[{'title': 'Pits of Hathsin', 'content': 'The ..."
2,Allomancy,"[Final Ascension, ref-wob-12692, ref-wob-16270...","[{'title': 'Allomancy', 'content': 'Allomancy ..."
3,Cosmere,"[ref-wob-4850, Sleepless, Herald, Ars Arcanum,...","[{'title': 'Cosmere', 'content': 'The cosmere ..."
4,Honor's Perpendicularity,"[Physical Realm, cosmere, Thaylen City, ref-bo...","[{'title': 'Honor's Perpendicularity', 'conten..."
5,Cephandrius,"[Category: Aliases, Hoid, Category: nominal re...",None
6,Hoid,"[ref-wob-3985, Sleepless, Design, AonDor, Cryp...","[{'title': 'Hoid', 'content': 'Hoid is a world..."


# Questions to ask
- How many total unique links in 6 pages?
- How many on average are book refs, or wob refs?
- How bad are the outliers?

# Goals
- Answer questions
- Trim down links
- Option to produce a list of links that are only the articles downloaded

In [53]:
from functools import reduce
from operator import iconcat
# Unique links
df['links'].dtypes = {list: str}
df_links = df['links'].copy()
df_links_stats = df.copy()
df_links_stats['num_links'] = df_links.apply(lambda x: len(x))
num_links = df_links_stats['num_links'].sum()
print(f"Total links: {num_links}")
flattened_links = reduce(iconcat, df_links, [])
print(len(flattened_links))
set_flattened = set(flattened_links)
unique_sum = len(set_flattened)
print(f"Unique links: {unique_sum}")
print(f"{round(unique_sum/num_links*100,2)}% overlap")

Total links: 1670
1670
Unique links: 1423
85.21% overlap


In [46]:
# % wob and book refs
# total refs
def get_ref_stats(row):
    total_refs = 0
    wob_refs = 0
    book_refs = 0
    other_refs = []
    for link in row['links']:
        if link.startswith("ref-"):
            total_refs += 1
            if link.startswith("ref-book"): book_refs += 1
            elif link.startswith("ref-wob"): wob_refs += 1
            else: other_refs.append(link)
    row['num_total_refs'] = total_refs
    row['num_book_refs'] = book_refs
    row['num_wob_refs'] = wob_refs
#     if total_refs > 0:
#         print(f"""{row['title']}
# Total refs: {total_refs}
# % wob: {(wob_refs/total_refs) * 100 if wob_refs > 0 else 0}
# % book: {(book_refs/total_refs) * 100 if book_refs > 0 else 0}
# """)
#     else:
#         print(f"{row['title']}\nTotal Refs: 0")
    return row
    
df_links_stats = df_links_stats.apply(lambda x: get_ref_stats(x), axis=1)
all_refs = df_links_stats['num_total_refs'].sum()
all_book_refs = df_links_stats['num_book_refs'].sum()
all_wob_refs = df_links_stats['num_wob_refs'].sum()

print(f"""Total Ref %
Total refs: {all_refs}
% wob: {round((all_wob_refs/all_refs) * 100,2)}
% book: {round((all_book_refs/all_refs) * 100,2)}
""")

Total Ref %
Total refs: 834
% wob: 53.24
% book: 46.76



In [59]:
df.dtypes

title       object
links       object
sections    object
dtype: object

In [58]:
p

[]